In [11]:
import numpy as np 
import cv2 

In [6]:
data = np.load("../data/running (1).npz")
blaze_pose = data['blaze_pose']
nturgb120 = data["nturgb120"]

In [7]:
blaze_pose.shape

(10, 60, 12, 2)

In [8]:
nturgb120.shape

(10, 90, 12, 2)

In [29]:
def gen_skeleton(frame, 
                 height,
                 width,
                 mapping_list = [(0, 1), (1, 3), (3, 5), 
                                 (0, 2), (2, 4), (0, 6), 
                                 (1, 7), (6, 7), (6, 8), 
                                 (7, 9), (8, 10), (9, 11)]):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles
    for i, coord in enumerate(frame):
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)
        img_3 = cv2.putText(img_3, f'{i}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
    # add lines
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img_3 = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3

def gen_video(points, 
              save_file, 
              frame_h, 
              frame_w, 
              is_3d=True,
              mapping_list = [(0, 1), (1, 3), (3, 5), 
                                 (0, 2), (2, 4), (0, 6), 
                                 (1, 7), (6, 7), (6, 8), 
                                 (7, 9), (8, 10), (9, 11)]):
    # make 3D if points are flatten
    if len(points.shape) == 2:
        if is_3d:
          fts = points.shape[1]
          x_cds = list(range(0, fts, 3))
          y_cds = list(range(1, fts, 3))
          z_cds = list(range(2, fts, 3))
          points = np.transpose(np.array([points[:, x_cds], 
                                          points[:, y_cds], 
                                          points[:, z_cds]]), (1,2,0))
        else:
          fts = points.shape[1]
          x_cds = list(range(0, fts, 2))
          y_cds = list(range(1, fts, 2))
          points = np.transpose(np.array([points[:, x_cds], 
                                          points[:, y_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    for __id,frame in enumerate(points):
        skel_image = gen_skeleton(frame, frame_h, frame_w,mapping_list=mapping_list)
        result.write(skel_image)

    result.release()

In [30]:
gen_video(blaze_pose[0, ...], '../data/blazepose_1.mp4', frame_h=768, frame_w=512, is_3d=False)

In [56]:
print(blaze_pose[1, 0, ...])

[[0.54797024 0.60294735]
 [0.5205617  0.59400249]
 [0.55808204 0.64982289]
 [0.50948101 0.61618036]
 [0.54365098 0.67001748]
 [0.49786344 0.63865703]
 [0.52284491 0.6459319 ]
 [0.5105415  0.62664694]
 [0.51880586 0.69901687]
 [0.49988976 0.62786216]
 [0.51609308 0.73129869]
 [0.4961406  0.640926  ]]


In [34]:
# index_map = [4,2,0,1,3,5,6,8,10,7,9,11]
index_map = [2,3,1,4,0,5,6,9,7,10,8,11]
repositioned_nturgbd120 = nturgb120[:, :, index_map, :]

In [31]:
gen_video(nturgb120[0, ...], '../data/nturgbd120_1.mp4', frame_h=768, frame_w=512, is_3d=False)

In [35]:
gen_video(repositioned_nturgbd120[0, ...], '../data/rep_nturgbd120_1.mp4', frame_h=768, frame_w=512, is_3d=False)

In [57]:
print(nturgb120[1, 0, ...])

[[0.44018828 0.59652898]
 [0.43288219 0.5395587 ]
 [0.42487099 0.46394463]
 [0.39120161 0.47150315]
 [0.38649667 0.5362738 ]
 [0.40198146 0.60957833]
 [0.4215476  0.59995435]
 [0.42048292 0.68824981]
 [0.41535443 0.76686593]
 [0.40649042 0.60542204]
 [0.40113177 0.6937612 ]
 [0.39785135 0.7749675 ]]


In [40]:
sample_arr = np.array([[1,2], [3,4], [5,6]])
sample_arr.shape

(3, 2)

In [38]:
sample_arr.reshape(-1)

array([1, 2, 3, 4, 5, 6])

---

In [41]:
from collections import defaultdict
from functools import reduce

In [42]:
skeleton_data = np.load('../data/skeleton_mediapipe.npz')
skeleton_classes, skeleton_mov = skeleton_data['arr_0'], skeleton_data['arr_1']
new_fts = [i for i in range(skeleton_mov.shape[-1]) if i%3 != 2]
seq_len = 60
skeleton_mov = skeleton_mov[:, :seq_len, new_fts]

skel_mean = defaultdict(list)
for i,c in enumerate(skeleton_classes):
    skel_mean[c] = skeleton_mov[i]

In [44]:
def order_skels(skel):
  joint_map = [4,2,0,1,3,5,6,8,10,7,9,11]
  index_map = reduce(lambda acc, ele : acc + [2*ele,2*ele+1], joint_map, [])
  ordered_skel = skel[:, :, index_map]
  return ordered_skel 

def reorder_skels(skel):
  joint_map = [2,3,1,4,0,5,6,9,7,10,8,11]
  index_map = reduce(lambda acc, ele : acc + [2*ele,2*ele+1], joint_map, [])
  reordered_skel = skel[:, :, index_map]
  return reordered_skel

In [45]:
skeleton_mov.shape

(177, 60, 24)

In [46]:
ordered_skel_mov = order_skels(skeleton_mov)
ordered_skel_mov.shape 

(177, 60, 24)

In [47]:
reordered_skel_mov = reorder_skels(ordered_skel_mov)
reordered_skel_mov.shape

(177, 60, 24)

In [52]:
mov1 = skeleton_mov[1, ...]
ord_mov1 = ordered_skel_mov[1, ...]
reord_mov1 = reordered_skel_mov[1, ...]

In [53]:
gen_video(mov1, '../data/blz_org_2.mp4', frame_h=768, frame_w=512, is_3d=False)

In [54]:
gen_video(ord_mov1, '../data/blz_ord_org_2.mp4', frame_h=768, frame_w=512, is_3d=False)

In [55]:
gen_video(reord_mov1, '../data/blz_reord_ord_org_2.mp4', frame_h=768, frame_w=512, is_3d=False)